In [76]:
import pandas as pd
from fuzzywuzzy import fuzz
from pandas import ExcelWriter

In [77]:
df = pd.read_excel("I.xlsx")
df.head(10)

,SpecID,QuestionText,ResponseBaseFldName,CreateDate
0,1991865953,Provide additional comments or clarifications ...,InAccCore9,2018-06-25 20:12:46.717
1,1955103041,"If applicable, provide the life coverage as a ...",InfLifeCoreAddl4,2018-06-21 17:53:29.397
2,1972152103,"If applicable, provide the life coverge as orp...",InfLifeCoreAddl7,2018-06-21 17:57:28.277
3,1991000950,Does your organization offer accident benefit ...,InvAccCore1,2018-06-25 17:36:15.167
4,1951233399,Indicate the type(s) of accident benefits prov...,InvAccCore1a,2018-06-25 18:57:26.423
5,1963767465,Indicate the type(s) of accident benefits cove...,InvAccCore2,2018-06-25 19:32:31.100
6,1968191132,"If applicable, provide the accident coverage a...",InvAccCore3,2018-06-25 19:42:32.507
7,1980394980,"If lump sum - variable amount, please describe.",InvAccCore4,2018-07-25 17:54:18.477
8,1970968411,"If applicable, provide the accident coverage a...",InvAccCore5,2018-06-25 19:52:27.400
9,1969774414,Indicate what elements of pay are used to dete...,InvAccCore6,2018-06-25 19:53:59.470


In [78]:
df["IsDeleted"] = 0

In [79]:
l=df.shape[0]

In [80]:
mydict={}
for index, row in df.iterrows():
    if index < l-1 and row.IsDeleted == 0:
        for i in range(index+1,l):
            if df.iloc[i,4] == 0:
                ratio = fuzz.token_set_ratio(row.QuestionText, df.iloc[i,1])
                if ratio > 70:
                    df.at[i,'IsDeleted'] = 1
                    df.at[index,'IsDeleted'] = 1
                    if df.iloc[index].SpecID in mydict:
                        mylist = mydict[df.iloc[index,0]]
                    else:
                        mylist = []
                    mylist.append(df.iloc[i,0])
                    mydict[row.SpecID] = mylist

In [81]:
list_dfs = []
for i, j in mydict.items():
    itemsToSelect=[]
    itemsToSelect.append(i)
    itemsToSelect = itemsToSelect+j
    df_subset = df.loc[df['SpecID'].isin(itemsToSelect)]
    list_dfs.append(df_subset)

In [82]:
def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()

In [83]:
save_xls(list_dfs,"Inv70.xls")